In [1]:
# Cell 1

import os
import csv
import math
import random
import numpy as np
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 멀티스레드(옵션)
torch.set_num_threads(8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

Using device: cuda


In [3]:
# Cell 2

class MovieLens:
    def __init__(self, ratings_path, movies_path=None):
        self.ratings_path = ratings_path
        self.movies_path  = movies_path or ""
        self.movieID_to_name = {}
        self.name_to_movieID = {}

    def load_ratings_df(self):
        import pandas as pd
        df = pd.read_csv(self.ratings_path, header=0, encoding='utf-8')
        return df

    def load_movies(self):
        """
        movies.csv -> movieID <-> name, etc. (옵션)
        """
        pass  # 생략

ratings_path = "/content/ratings.csv"
ml = MovieLens(ratings_path)
df = ml.load_ratings_df()

# timestamp 열이 있다고 가정
df = df.sort_values('timestamp').reset_index(drop=True)

cutoff_idx= int(len(df)*0.8)
train_df  = df.iloc[:cutoff_idx]
test_df   = df.iloc[cutoff_idx:]

def build_user_sequences(df_):
    from collections import defaultdict
    seq_dict= defaultdict(list)
    for row in df_.itertuples():
        # (Index, userId, movieId, rating, timestamp)
        user   = int(row.userId)
        item   = int(row.movieId)
        rating = float(row.rating)
        seq_dict[user].append((item, rating))
    return seq_dict

user_seq_train = build_user_sequences(train_df)
user_seq_test  = build_user_sequences(test_df)

all_users = set(user_seq_train.keys()).union(user_seq_test.keys())
all_items = set()
for u, seq in user_seq_train.items():
    for (i,r) in seq:
        all_items.add(i)
for u, seq in user_seq_test.items():
    for (i,r) in seq:
        all_items.add(i)

num_users= max(all_users)+1
num_items= max(all_items)+1

print("Train user-seq:", len(user_seq_train), "Test user-seq:", len(user_seq_test))
print("num_users:", num_users, "num_items:", num_items)

Train user-seq: 547 Test user-seq: 147
num_users: 672 num_items: 163950


In [4]:
# Cell 3

from torch.utils.data import Dataset, DataLoader

class UserSequenceDataset(Dataset):
    def __init__(self, user_sequences, max_seq_len=10):
        self.samples=[]
        for u, seq in user_sequences.items():
            if len(seq)<2:
                continue
            start=0
            while start<len(seq):
                end= min(start+max_seq_len, len(seq))
                sub_seq= seq[start:end]
                if len(sub_seq)>=2:
                    items   = [x[0] for x in sub_seq]
                    ratings = [x[1] for x in sub_seq]
                    self.samples.append((u, items, ratings))
                start=end

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

def collate_fn(batch):
    users, items_list, ratings_list, lengths= [], [], [], []
    max_len=0
    for (u,it,rt) in batch:
        if len(it)>max_len:
            max_len=len(it)

    for (u,it,rt) in batch:
        seq_len= len(it)
        lengths.append(seq_len)
        users.append(u)

        pad_it= it + [0]*(max_len-seq_len)
        pad_rt= rt + [0.0]*(max_len-seq_len)

        items_list.append(pad_it)
        ratings_list.append(pad_rt)

    users_tensor   = torch.LongTensor(users)
    items_tensor   = torch.LongTensor(items_list)
    ratings_tensor = torch.FloatTensor(ratings_list)
    lengths_tensor = torch.LongTensor(lengths)
    return (users_tensor, items_tensor, ratings_tensor, lengths_tensor)

In [5]:
# Cell 4

class GRUCellBasic(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.W_z = nn.Linear(input_size+hidden_size, hidden_size)
        self.W_r = nn.Linear(input_size+hidden_size, hidden_size)
        self.W_h = nn.Linear(input_size+hidden_size, hidden_size)

    def forward(self, x, h_prev):
        combined= torch.cat([x,h_prev],dim=1)
        z= torch.sigmoid(self.W_z(combined))
        r= torch.sigmoid(self.W_r(combined))
        comb_r= torch.cat([x, r*h_prev],dim=1)
        h_tilde= torch.tanh(self.W_h(comb_r))
        h= (1-z)*h_prev + z*h_tilde
        return h

class GRUCellModified(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.W_z = nn.Linear(input_size+hidden_size, hidden_size)
        self.W_r = nn.Linear(input_size+hidden_size, hidden_size)
        self.W_h = nn.Linear(input_size+hidden_size, hidden_size)
        self.lrelu = nn.LeakyReLU(0.2)

    def forward(self, x, h_prev):
        combined= torch.cat([x,h_prev],dim=1)
        z_pre= self.W_z(combined)
        z= self.lrelu(z_pre)
        r= torch.sigmoid(self.W_r(combined))
        comb_r= torch.cat([x, r*h_prev],dim=1)
        h_tilde= torch.tanh(self.W_h(comb_r))
        h= (1-z)*h_prev + z*h_tilde
        return h

class RRecGAN_Generator(nn.Module):
    def __init__(self, num_users, num_items, embed_dim, hidden_size, gru_type='basic'):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embed_dim, padding_idx=0)
        self.item_embedding = nn.Embedding(num_items, embed_dim, padding_idx=0)
        self.rating_embedding= nn.Linear(1, embed_dim)

        input_size= embed_dim*3
        if gru_type=='basic':
            self.gru_cell= GRUCellBasic(input_size, hidden_size)
        else:
            self.gru_cell= GRUCellModified(input_size, hidden_size)

        self.output_layer= nn.Linear(hidden_size,1)

    def forward(self, users, items, ratings, lengths):
        device= items.device
        batch_size, seq_len= items.size()
        h= torch.zeros(batch_size, self.output_layer.in_features, device=device)

        user_emb= self.user_embedding(users)
        preds=[]
        for t in range(seq_len):
            it= items[:,t]
            rt= ratings[:,t].unsqueeze(-1)
            it_emb= self.item_embedding(it)
            rt_emb= self.rating_embedding(rt)
            x= torch.cat([user_emb, it_emb, rt_emb],dim=1)
            h= self.gru_cell(x,h)
            pred= self.output_layer(h)
            preds.append(pred)
        preds= torch.cat(preds, dim=1)
        return preds

class RRecGAN_Discriminator(nn.Module):
    def __init__(self, num_users, num_items, embed_dim, hidden_size, gru_type='basic'):
        super().__init__()
        self.user_embedding= nn.Embedding(num_users, embed_dim, padding_idx=0)
        self.item_embedding= nn.Embedding(num_items, embed_dim, padding_idx=0)
        self.rating_embedding= nn.Linear(1, embed_dim)

        input_size= embed_dim*3
        if gru_type=='basic':
            self.gru_cell= GRUCellBasic(input_size, hidden_size)
        else:
            self.gru_cell= GRUCellModified(input_size, hidden_size)

        self.output_layer= nn.Linear(hidden_size,1)

    def forward(self, users, items, ratings, lengths):
        device= items.device
        batch_size, seq_len= items.size()
        h= torch.zeros(batch_size, self.output_layer.in_features, device=device)

        user_emb= self.user_embedding(users)
        for t in range(seq_len):
            it= items[:,t]
            rt= ratings[:,t].unsqueeze(-1)
            it_emb= self.item_embedding(it)
            rt_emb= self.rating_embedding(rt)
            x= torch.cat([user_emb, it_emb, rt_emb],dim=1)
            h= self.gru_cell(x,h)
        logit= self.output_layer(h)
        out= torch.sigmoid(logit)
        return out

In [6]:
# Cell 5

def train_rrecgan(generator, discriminator, train_loader,
                  num_epochs=15,
                  g_lr=5e-4,
                  d_lr=5e-4,
                  lambda_mse=0.1,
                  grad_clip=10.0,
                  device='cuda',
                  S_g=1, S_d=1):
    import torch.nn.utils as nn_utils

    gen_opt= optim.Adam(generator.parameters(), lr=g_lr)
    dis_opt= optim.Adam(discriminator.parameters(), lr=d_lr)

    bce_loss= nn.BCELoss()
    mse_loss= nn.MSELoss()

    generator.to(device)
    discriminator.to(device)

    for epoch in range(num_epochs):
        generator.train()
        discriminator.train()

        epoch_g_loss=0.0
        epoch_d_loss=0.0
        batch_count=0

        for (users, items, ratings, lengths) in train_loader:
            users   = users.to(device)
            items   = items.to(device)
            ratings = ratings.to(device)
            lengths = lengths.to(device)

            batch_size= items.size(0)

            # (1) Generator update
            for _ in range(S_g):
                gen_opt.zero_grad()
                gen_ratings= generator(users, items, ratings, lengths)
                d_out_gen= discriminator(users, items, gen_ratings, lengths)
                real_labels= torch.ones(batch_size,1,device=device)

                gan_loss= bce_loss(d_out_gen, real_labels)
                rec_loss= mse_loss(gen_ratings, ratings)
                g_loss= gan_loss + lambda_mse*rec_loss

                g_loss.backward()
                nn_utils.clip_grad_norm_(generator.parameters(), grad_clip)
                gen_opt.step()

            # (2) Discriminator update
            for _ in range(S_d):
                dis_opt.zero_grad()
                # real
                real_labels= torch.ones(batch_size,1,device=device)
                d_out_real= discriminator(users, items, ratings, lengths)
                d_loss_real= bce_loss(d_out_real, real_labels)

                # fake
                with torch.no_grad():
                    fake_ratings= generator(users, items, ratings, lengths)
                fake_labels= torch.zeros(batch_size,1,device=device)
                d_out_fake= discriminator(users, items, fake_ratings, lengths)
                d_loss_fake= bce_loss(d_out_fake, fake_labels)

                d_loss= d_loss_real + d_loss_fake
                d_loss.backward()
                nn_utils.clip_grad_norm_(discriminator.parameters(), grad_clip)
                dis_opt.step()

            epoch_g_loss+= g_loss.item()
            epoch_d_loss+= d_loss.item()
            batch_count+=1

        avg_g= epoch_g_loss/batch_count if batch_count>0 else 0.0
        avg_d= epoch_d_loss/batch_count if batch_count>0 else 0.0
        print(f"[Epoch {epoch+1}/{num_epochs}] G_loss={avg_g:.4f}, D_loss={avg_d:.4f}")

    print("학습 완료!")
    return generator, discriminator

In [8]:
# Cell 6

import math
import numpy as np
from collections import defaultdict

########################
## 0) RMSE, MAE
########################
def rmse(predictions, targets):
    return math.sqrt(np.mean((np.array(predictions)-np.array(targets))**2))

def mae(predictions, targets):
    return np.mean(np.abs(np.array(predictions)-np.array(targets)))

########################
## 1) 사용자별 Top-N
########################
def get_topN_for_all_users_recgan(model, all_users, all_items, user_items_dict,
                                  popularity_ranks,
                                  top_k_candidates=2000,
                                  N=10):
    """
     - 이미 평가한 아이템 제외
     - 인기도 rank 기반 pruning (top_k_candidates)
     - model (RecGAN Generator)로 (u,i) 평점 예측
    """
    user_topN= {}
    model.eval()

    for u in all_users:
        rated_items = user_items_dict[u]
        # 후보군: all_items - rated_items
        cands= [i for i in all_items if i not in rated_items]

        # 인기 높은 아이템만 top_k_candidates개
        if len(cands)> top_k_candidates:
            cands= sorted(cands, key=lambda x: popularity_ranks.get(x,9999999))
            cands= cands[:top_k_candidates]

        scores=[]
        for i in cands:
            user_tensor   = torch.LongTensor([u]).to(device)
            item_tensor   = torch.LongTensor([[i]]).to(device)
            rating_tensor = torch.FloatTensor([[0.0]]).to(device)
            length_tensor = torch.LongTensor([1]).to(device)

            with torch.no_grad():
                pr_2d = model(user_tensor, item_tensor, rating_tensor, length_tensor)
            pred_val= pr_2d.item()
            scores.append((i,pred_val))

        scores.sort(key=lambda x: x[1], reverse=True)
        top_items= [x[0] for x in scores[:N]]
        user_topN[u] = top_items

    return user_topN

########################
## 2) HR, cHR, AHAR
########################
def evaluate_topN_metrics(user_topN, test_data, rating_threshold=4.0, N=10):
    """
    test_data: [(u,i,r), ...]
    user_topN: {u: [item1, item2, ... itemN]}
    - HR: hit ratio
    - cHR
    - AHAR
    """
    hits_hr=0
    total_hr=0
    user_hits_count= defaultdict(int)
    user_above_thr=  defaultdict(int)

    # user_topN_rank
    user_topN_rank= {}
    for u, items in user_topN.items():
        rank_map={}
        for idx,it in enumerate(items):
            rank_map[it]= idx+1
        user_topN_rank[u]= rank_map

    sum_ranks=0
    count_ranks=0

    for (u,i,r) in test_data:
        if r>= rating_threshold:
            total_hr+=1
            if i in user_topN[u]:
                hits_hr+=1

            user_above_thr[u]+=1
            rank_i= user_topN_rank[u].get(i, N+1)
            sum_ranks+= rank_i
            count_ranks+=1

            if rank_i<=N:
                user_hits_count[u]+=1

    hr_val= hits_hr/ total_hr if total_hr>0 else 0.0
    total_users_chr= sum(1 for k in user_above_thr if user_above_thr[k]>0)
    total_hits_chr= sum(user_hits_count.values())
    chr_val= total_hits_chr/total_users_chr if total_users_chr>0 else 0.0
    ahar_val= sum_ranks/count_ranks if count_ranks>0 else 0.0

    return hr_val, chr_val, ahar_val

########################
## 3) Coverage, Diversity, Novelty
########################
def coverage_with_topN(user_topN, all_items):
    recommended= set()
    for u, top_items in user_topN.items():
        for it in top_items:
            recommended.add(it)
    return len(recommended)/ len(all_items) if len(all_items)>0 else 0.0

def diversity_with_topN(user_topN, movie_genres):
    distances=[]
    for u, top_items in user_topN.items():
        if len(top_items)<2:
            continue
        pair_sum=0.0
        pair_count=0
        for i1 in range(len(top_items)):
            for i2 in range(i1+1, len(top_items)):
                it1= top_items[i1]
                it2= top_items[i2]
                g1= movie_genres.get(it1,[])
                g2= movie_genres.get(it2,[])
                if len(g1)== len(g2):
                    inter=0
                    union=0
                    for x,y in zip(g1,g2):
                        if x==1 and y==1:
                            inter+=1
                        if x==1 or y==1:
                            union+=1
                    if union>0:
                        jacc_dist= 1-(inter/union)
                        pair_sum+= jacc_dist
                        pair_count+=1
        if pair_count>0:
            distances.append(pair_sum/pair_count)
    if len(distances)==0:
        return 0.0
    import numpy as np
    return float(np.mean(distances))

def novelty_with_topN(user_topN, popularity_ranks):
    ranks=[]
    for u, top_items in user_topN.items():
        for it in top_items:
            ranks.append(popularity_ranks.get(it,9999999))
    if len(ranks)==0:
        return 0.0
    import numpy as np
    return float(np.mean(ranks))

########################
## 4) 최종 평가
########################
def evaluate_model_rbm_style_for_recgan(generator,
                                        train_data, test_data,   # (u,i,r) list
                                        all_users, all_items,
                                        user_items_dict,
                                        popularity_ranks,
                                        movie_genres,
                                        top_k_candidates=2000,
                                        N=10,
                                        rating_threshold=4.0):
    """
    1) RMSE, MAE
    2) 사용자별 TopN -> HR, cHR, AHAR
    3) Coverage, Diversity(장르 Jaccard), Novelty(인기도 rank)
    """
    # 1) RMSE, MAE
    generator.eval()
    preds, trues= [], []
    for (u,i,r) in test_data:
        user_tensor   = torch.LongTensor([u]).to(device)
        item_tensor   = torch.LongTensor([[i]]).to(device)
        rating_tensor = torch.FloatTensor([[0.0]]).to(device)
        length_tensor = torch.LongTensor([1]).to(device)

        with torch.no_grad():
            pr_2d= generator(user_tensor, item_tensor, rating_tensor, length_tensor)
        preds.append(pr_2d.item())
        trues.append(r)

    RMSE_val= rmse(preds, trues)
    MAE_val = mae(preds, trues)

    # 2) TopN
    user_topN= get_topN_for_all_users_recgan(
        model=generator,
        all_users=all_users,
        all_items=all_items,
        user_items_dict=user_items_dict,
        popularity_ranks=popularity_ranks,
        top_k_candidates=top_k_candidates,
        N=N
    )

    # 3) HR, cHR, AHAR
    hr_val, chr_val, ahar_val= evaluate_topN_metrics(user_topN, test_data,
                                                     rating_threshold=rating_threshold, N=N)

    # 4) Coverage, Diversity, Novelty
    coverage_val= coverage_with_topN(user_topN, all_items)
    diversity_val= diversity_with_topN(user_topN, movie_genres)
    novelty_val= novelty_with_topN(user_topN, popularity_ranks)

    return {
        "RMSE": RMSE_val,
        "MAE": MAE_val,
        "HR": hr_val,
        "cHR": chr_val,
        "AHAR": ahar_val,
        "Coverage": coverage_val,
        "Diversity": diversity_val,
        "Novelty": novelty_val
    }

In [9]:
# Cell 7

# (A) (u,i,r) 형태로 train_data, test_data 리스트 구성
train_data_list= []
for u, seq in user_seq_train.items():
    for (i,r) in seq:
        train_data_list.append((u,i,r))

test_data_list= []
for u, seq in user_seq_test.items():
    for (i,r) in seq:
        test_data_list.append((u,i,r))

# 사용자별 이미 본 아이템
user_items_dict= defaultdict(set)
for (u,i,r) in train_data_list:
    user_items_dict[u].add(i)

# 장르, 인기도 rank (RBM 코드 비슷하게)
def get_movie_genres():
    """
    장르 비트필드. (예시)
    실제로는 movies.csv를 parse하여 genre -> bitfield
    여기서는 임시로 빈 dict or random
    """
    return defaultdict(list)

def get_popularity_ranks():
    """
    아이템 인기도 rank(1=가장 인기)
    """
    from collections import Counter
    c= Counter()
    for (u,i,r) in train_data_list:
        c[i]+=1
    sorted_items= sorted(c.items(), key=lambda x:x[1], reverse=True)
    rank_dict= {}
    rank=1
    for (it,cnt) in sorted_items:
        rank_dict[it]= rank
        rank+=1
    return rank_dict

movie_genres= get_movie_genres()  # 실제로는 movies.csv에서 장르 파싱
popularity_ranks= get_popularity_ranks()

# DataLoader
train_dataset= UserSequenceDataset(user_seq_train, max_seq_len=10)
train_loader= DataLoader(train_dataset, batch_size=512, shuffle=True,
                         collate_fn=collate_fn, drop_last=False)

# RecGAN1
gen1= RRecGAN_Generator(num_users, num_items, embed_dim=128, hidden_size=128, gru_type='basic')
dis1= RRecGAN_Discriminator(num_users, num_items, embed_dim=128, hidden_size=128, gru_type='basic')

print("\n===== RecGAN1 (Basic GRU) 학습 =====")
gen1, dis1= train_rrecgan(
    generator=gen1,
    discriminator=dis1,
    train_loader=train_loader,
    num_epochs=10,      # 실제론 15~20+ / bigger hidden etc
    g_lr=5e-4,
    d_lr=5e-4,
    lambda_mse=0.1,
    grad_clip=5.0,
    device=device,
    S_g=1,
    S_d=1
)

# 평가
metrics1= evaluate_model_rbm_style_for_recgan(
    generator=gen1,
    train_data= train_data_list,
    test_data=  test_data_list,
    all_users= set(u for (u,i,r) in train_data_list+test_data_list),
    all_items= set(i for (u,i,r) in train_data_list+test_data_list),
    user_items_dict= user_items_dict,
    popularity_ranks= popularity_ranks,
    movie_genres= movie_genres,
    top_k_candidates=2000,
    N=10,
    rating_threshold=4.0
)
print("\n===== Evaluate RecGAN1 (RBM Style) =====")
for k,v in metrics1.items():
    print(f"{k} = {v:.4f}")

# RecGAN2
gen2= RRecGAN_Generator(num_users, num_items, embed_dim=128, hidden_size=128, gru_type='modified')
dis2= RRecGAN_Discriminator(num_users, num_items, embed_dim=128, hidden_size=128, gru_type='modified')

print("\n===== RecGAN2 (Modified GRU) 학습 =====")
gen2, dis2= train_rrecgan(
    generator=gen2,
    discriminator=dis2,
    train_loader=train_loader,
    num_epochs=10,
    g_lr=5e-4,
    d_lr=5e-4,
    lambda_mse=0.1,
    grad_clip=5.0,
    device=device,
    S_g=1,
    S_d=1
)

metrics2= evaluate_model_rbm_style_for_recgan(
    generator=gen2,
    train_data= train_data_list,
    test_data=  test_data_list,
    all_users= set(u for (u,i,r) in train_data_list+test_data_list),
    all_items= set(i for (u,i,r) in train_data_list+test_data_list),
    user_items_dict= user_items_dict,
    popularity_ranks= popularity_ranks,
    movie_genres= movie_genres,
    top_k_candidates=2000,
    N=10,
    rating_threshold=4.0
)
print("\n===== Evaluate RecGAN2 (RBM Style) =====")
for k,v in metrics2.items():
    print(f"{k} = {v:.4f}")


===== RecGAN1 (Basic GRU) 학습 =====
[Epoch 1/10] G_loss=1.0259, D_loss=1.4997
[Epoch 2/10] G_loss=0.9163, D_loss=1.4280
[Epoch 3/10] G_loss=0.7605, D_loss=1.3828
[Epoch 4/10] G_loss=0.6835, D_loss=1.3808
[Epoch 5/10] G_loss=0.7850, D_loss=1.4582
[Epoch 6/10] G_loss=0.7143, D_loss=1.3740
[Epoch 7/10] G_loss=0.7181, D_loss=1.3777
[Epoch 8/10] G_loss=0.7079, D_loss=1.3962
[Epoch 9/10] G_loss=0.7265, D_loss=1.3846
[Epoch 10/10] G_loss=0.6939, D_loss=1.3870
학습 완료!

===== Evaluate RecGAN1 (RBM Style) =====
RMSE = 3.6920
MAE = 3.5341
HR = 0.0087
cHR = 0.5918
AHAR = 10.9523
Coverage = 0.0122
Diversity = 0.0000
Novelty = 910.9858

===== RecGAN2 (Modified GRU) 학습 =====
[Epoch 1/10] G_loss=1.4408, D_loss=1.2037
[Epoch 2/10] G_loss=2.5654, D_loss=1.9370
[Epoch 3/10] G_loss=1.0821, D_loss=1.4451
[Epoch 4/10] G_loss=1827.6258, D_loss=1.4616
[Epoch 5/10] G_loss=0.9232, D_loss=1.4313
[Epoch 6/10] G_loss=0.6735, D_loss=1.4171
[Epoch 7/10] G_loss=0.7513, D_loss=1.3983
[Epoch 8/10] G_loss=0.7737, D_loss=

In [10]:
# Cell 8

def recommend_for_user(model, user_id, all_items, user_items_dict,
                       popularity_ranks, N=10, top_k_candidates=2000):
    """
    특정 user에 대해서만
     - 이미 평가한 아이템 제외
     - 인기도 rank 기반 후보 제한
     - RecGAN 모델로 평점 예측 -> 상위 N
    """
    rated_items= user_items_dict[user_id]
    cands= [i for i in all_items if i not in rated_items]
    if len(cands)> top_k_candidates:
        cands= sorted(cands, key=lambda x: popularity_ranks.get(x,9999999))
        cands= cands[:top_k_candidates]

    scores=[]
    model.eval()
    with torch.no_grad():
        for it_ in cands:
            user_tensor   = torch.LongTensor([user_id]).to(device)
            item_tensor   = torch.LongTensor([[it_]]).to(device)
            rating_tensor = torch.FloatTensor([[0.0]]).to(device)
            length_tensor = torch.LongTensor([1]).to(device)

            pr_2d= model(user_tensor, item_tensor, rating_tensor, length_tensor)
            scores.append((it_, pr_2d.item()))
    scores.sort(key=lambda x:x[1], reverse=True)
    top_items= [x[0] for x in scores[:N]]
    return top_items


def print_recommendations_for_user(model, user_id,
                                   all_items, user_items_dict,
                                   popularity_ranks,
                                   ml_object,  # MovieLens 객체 (movieID->name 매핑)
                                   N=10):
    """
    user_id에게 추천된 아이템 N개를 영화 제목과 함께 출력
    """
    top_items= recommend_for_user(model, user_id, all_items, user_items_dict,
                                  popularity_ranks, N=N)
    print(f"\n=== 추천 결과 for User {user_id} ===")
    for rank_idx, item_id in enumerate(top_items, start=1):
        movie_name= ml_object.movieID_to_name.get(item_id, f"Movie-{item_id}")
        print(f"{rank_idx}. item_id={item_id}, title={movie_name}")


# 예시: RecGAN2 모델로, user 10에게 추천된 상위 10개 영화 출력
some_user= 10
print_recommendations_for_user(
    model=gen2,        # RecGAN2 (Modified GRU)
    user_id= some_user,
    all_items= set(i for (u,i,r) in train_data_list+test_data_list),
    user_items_dict= user_items_dict,
    popularity_ranks= popularity_ranks,
    ml_object= ml,     # MovieLens 객체( movieID_to_name 이용 )
    N=10
)


=== 추천 결과 for User 10 ===
1. item_id=2369, title=Movie-2369
2. item_id=938, title=Movie-938
3. item_id=3617, title=Movie-3617
4. item_id=3308, title=Movie-3308
5. item_id=2058, title=Movie-2058
6. item_id=3996, title=Movie-3996
7. item_id=1131, title=Movie-1131
8. item_id=674, title=Movie-674
9. item_id=1214, title=Movie-1214
10. item_id=613, title=Movie-613
